# Kapitel 7: Chat-Anwendungen erstellen
## Github Models API Schnellstart

Dieses Notebook wurde aus dem [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) übernommen, das Notebooks enthält, die auf [Azure OpenAI](notebook-azure-openai.ipynb) Dienste zugreifen.


# Überblick  
„Große Sprachmodelle sind Funktionen, die Text auf Text abbilden. Wenn sie eine Eingabezeichenkette erhalten, versuchen sie vorherzusagen, welcher Text als Nächstes kommt“(1). Dieses „Schnellstart“-Notebook führt Nutzer in grundlegende LLM-Konzepte ein, zeigt die wichtigsten Paketvoraussetzungen für den Einstieg mit AML, gibt eine sanfte Einführung in das Prompt-Design und enthält mehrere kurze Beispiele für verschiedene Anwendungsfälle.


## Inhaltsverzeichnis  

[Überblick](../../../../07-building-chat-applications/python)  
[Wie man den OpenAI Service verwendet](../../../../07-building-chat-applications/python)  
[1. Erstellen Ihres OpenAI Service](../../../../07-building-chat-applications/python)  
[2. Installation](../../../../07-building-chat-applications/python)    
[3. Zugangsdaten](../../../../07-building-chat-applications/python)  

[Anwendungsfälle](../../../../07-building-chat-applications/python)    
[1. Text zusammenfassen](../../../../07-building-chat-applications/python)  
[2. Text klassifizieren](../../../../07-building-chat-applications/python)  
[3. Neue Produktnamen generieren](../../../../07-building-chat-applications/python)  
[4. Einen Klassifikator feinabstimmen](../../../../07-building-chat-applications/python)  

[Referenzen](../../../../07-building-chat-applications/python)


### Erstellen Sie Ihren ersten Prompt  
Diese kurze Übung bietet eine grundlegende Einführung, wie Sie Prompts an ein Modell in Github Models für eine einfache Aufgabe wie die „Zusammenfassung“ senden können.

**Schritte**:  
1. Installieren Sie die Bibliothek `azure-ai-inference` in Ihrer Python-Umgebung, falls Sie dies noch nicht getan haben.  
2. Laden Sie die Standard-Hilfsbibliotheken und richten Sie die Github Models-Anmeldedaten ein.  
3. Wählen Sie ein Modell für Ihre Aufgabe aus  
4. Erstellen Sie einen einfachen Prompt für das Modell  
5. Senden Sie Ihre Anfrage an die Modell-API!


### 1. Installieren Sie `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Das richtige Modell finden  
Die Modelle GPT-3.5-turbo oder GPT-4 können natürliche Sprache verstehen und erzeugen.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Prompt-Design  

„Das Besondere an großen Sprachmodellen ist, dass sie durch das Minimieren des Vorhersagefehlers über riesige Mengen an Text letztlich Konzepte lernen, die für diese Vorhersagen nützlich sind. Zum Beispiel lernen sie Konzepte wie“(1):

* wie man richtig schreibt
* wie Grammatik funktioniert
* wie man umformuliert
* wie man Fragen beantwortet
* wie man ein Gespräch führt
* wie man in vielen Sprachen schreibt
* wie man programmiert
* usw.

#### Wie man ein großes Sprachmodell steuert  
„Von allen Eingaben für ein großes Sprachmodell ist der Text-Prompt mit Abstand der wichtigste Faktor(1).

Große Sprachmodelle können auf verschiedene Arten dazu gebracht werden, eine Ausgabe zu erzeugen:

Anweisung: Sage dem Modell, was du möchtest
Vervollständigung: Bringe das Modell dazu, den Anfang dessen zu vervollständigen, was du möchtest
Demonstration: Zeige dem Modell, was du möchtest, entweder durch:
Einige Beispiele im Prompt
Viele hunderte oder tausende Beispiele in einem Fine-Tuning-Trainingsdatensatz“



#### Es gibt drei grundlegende Richtlinien für die Erstellung von Prompts:

**Zeigen und erklären**. Mache klar, was du möchtest – entweder durch Anweisungen, Beispiele oder eine Kombination aus beidem. Wenn du möchtest, dass das Modell eine Liste alphabetisch sortiert oder einen Absatz nach Stimmung klassifiziert, zeige ihm, dass genau das gewünscht ist.

**Gib hochwertige Daten vor**. Wenn du versuchst, einen Klassifizierer zu bauen oder das Modell einem Muster folgen zu lassen, stelle sicher, dass genügend Beispiele vorhanden sind. Überprüfe deine Beispiele sorgfältig – das Modell ist meist klug genug, um einfache Rechtschreibfehler zu erkennen und trotzdem zu antworten, aber es könnte auch annehmen, dass dies Absicht ist, was die Antwort beeinflussen kann.

**Überprüfe deine Einstellungen.** Die Einstellungen für Temperatur und top_p steuern, wie vorhersehbar das Modell bei der Antwort ist. Wenn du eine Antwort erwartest, bei der es nur eine richtige Lösung gibt, solltest du diese Werte niedriger setzen. Wenn du vielfältigere Antworten möchtest, kannst du sie höher einstellen. Der häufigste Fehler bei diesen Einstellungen ist, zu glauben, dass sie die „Intelligenz“ oder „Kreativität“ des Modells steuern.


Quelle: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Text zusammenfassen  
#### Herausforderung  
Fasse einen Text zusammen, indem du am Ende eines Textabschnitts ein 'tl;dr:' hinzufügst. Beachte, wie das Modell in der Lage ist, verschiedene Aufgaben ohne zusätzliche Anweisungen auszuführen. Du kannst mit ausführlicheren Prompts als tl;dr experimentieren, um das Verhalten des Modells zu beeinflussen und die Zusammenfassung nach deinen Wünschen anzupassen(3).  

Aktuelle Arbeiten haben gezeigt, dass durch Vortraining auf einem großen Textkorpus und anschließendes Feintuning auf eine spezifische Aufgabe deutliche Fortschritte bei vielen NLP-Aufgaben und Benchmarks erzielt werden können. Obwohl diese Methode in der Regel aufgabenunabhängig in der Architektur ist, benötigt sie dennoch aufgabenspezifische Feintuning-Datensätze mit Tausenden oder Zehntausenden von Beispielen. Im Gegensatz dazu können Menschen in der Regel eine neue Sprachaufgabe schon mit wenigen Beispielen oder einfachen Anweisungen bewältigen – etwas, womit aktuelle NLP-Systeme noch weitgehend Schwierigkeiten haben. Hier zeigen wir, dass die Skalierung von Sprachmodellen die aufgabenunabhängige Few-Shot-Leistung deutlich verbessert und manchmal sogar mit bisherigen State-of-the-Art-Feintuning-Ansätzen konkurrieren kann.  

Tl;dr


# Übungen für verschiedene Anwendungsfälle  
1. Text zusammenfassen  
2. Text klassifizieren  
3. Neue Produktnamen generieren


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Text klassifizieren  
#### Herausforderung  
Ordne Elemente in Kategorien ein, die zur Laufzeit vorgegeben werden. Im folgenden Beispiel geben wir sowohl die Kategorien als auch den zu klassifizierenden Text im Prompt an (*playground_reference).

Kundenanfrage: Hallo, eine der Tasten auf meiner Laptop-Tastatur ist kürzlich kaputt gegangen und ich brauche einen Ersatz:

Klassifizierte Kategorie:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Neue Produktnamen generieren
#### Herausforderung
Erstelle Produktnamen aus Beispielwörtern. In der Aufgabenstellung geben wir Informationen über das Produkt, für das Namen generiert werden sollen. Außerdem zeigen wir ein ähnliches Beispiel, um das gewünschte Muster zu verdeutlichen. Die Temperatur wurde hoch eingestellt, um mehr Zufall und innovative Antworten zu ermöglichen.

Produktbeschreibung: Ein Milchshake-Maker für zuhause
Stichwörter: schnell, gesund, kompakt.
Produktnamen: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Produktbeschreibung: Ein Paar Schuhe, das auf jede Fußgröße passt.
Stichwörter: anpassbar, passgenau, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referenzen  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Beispiele](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Best Practices für das Fine-Tuning von GPT-3 zur Textklassifizierung](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Für weitere Hilfe  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Mitwirkende
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
